In [ ]:
from pynetdicom import AE
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelFind
from pydicom.dataset import Dataset
import logging

In [ ]:
dcmtk_folder = 'dcmtk-3.6.0-win32-i386\bin\';
output_file = 'log.txt';

dcmaet = 'CIVILISTENUC';
%dcmaec = 'algoFIR';
dcmaec = 'csps1FIR';
dcmaem = 'CIVILISTENUC';
dcmhost = '155.105.3.105';
%dcmhost = '155.105.54.78';
dcmport = '104';
incoming_port = '104';

opcommand1 = [dcmtk_folder,'movescu.exe -aet ',dcmaet,' -aec ',dcmaec,' -aem ',dcmaem,' -od ', output_folder_path];
opcommand2 = [' -v -P -k QueryRetrieveLevel=IMAGE -k Modality=',modality,' -k StudyDate=',date];
opcommand3 = [' -k InstanceNumber=',instanceNumber,' -k SeriesInstanceUID=',seriesInstanceUID,' -k ModalitiesInStudy=',modalitiesInStudies];
opcommand4 = [' ',dcmhost,' ',dcmport,' --port ',incoming_port];

In [ ]:
ae_tile = b'CIVILISTENUC'
ae_called_title = b'csps1FIR';
host = '155.105.3.105'
port = 104

ae = AE(ae_title=ae_tile)
ae.add_requested_context(PatientRootQueryRetrieveInformationModelFind)

# Create our Identifier (query) dataset
ds = Dataset()
ds.StudyDate = '20180622'
ds.PatientID = '2718821'
ds.QueryRetrieveLevel = 'PATIENT'

# Associate with peer AE
assoc = ae.associate(host, port, ae_title=ae_called_title)

if assoc.is_established:
    # Use the C-FIND service to send the identifier
    responses = assoc.send_c_find(ds, PatientRootQueryRetrieveInformationModelFind)

    for (status, identifier) in responses:
        if status:
            print('C-FIND query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then identifier is the C-FIND response
            if status.Status in (0xFF00, 0xFF01):
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

     # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

In [ ]:
assoc

In [ ]:
assoc.requestor.primitive